# Recreate Figure 2H

In [ ]:
library(fgsea)
library(extrafont)
library(cowplot)
library(ggpubr)
library(ggrepel)
library(dplyr)
library(gplots)
library(RColorBrewer)
library(tibble)

In [ ]:
## Import differentially expressed gene lists
mono_bvc_f <- read.table("Aug20_BLZ_data/Mono-CSF1Ri_vs_Mono-Vehicle_fdr05_Aug20.csv", sep="\t", header=F)
mono_bvc_f <- deframe(mono_bvc_f)
mac_bvc_f <- read.table("Aug20_BLZ_data/Mac-CSF1Ri_vs_Mac-Vehicle_fdr05_Aug20.csv", sep="\t", header=F)
mac_bvc_f <- deframe(mac_bvc_f)

In [ ]:
## Import all GO BP pathways and genes
gofile <- scan("GOids_genes.txt", what="", sep="\n")   
pathways_all <- strsplit(gofile, "\t")

# Extract the first vector element and set it as the list element name
names(pathways_all) <- sapply(pathways_all, `[[`, 1)
# Remove the first vector element from each list element
pathways_all <- lapply(pathways_all, `[`, -1)

In [ ]:
# fgsea - only retain pathways where 15+ genes expressed and no more than 300
set.seed(1234)
mono_res <- fgsea(pathways=pathways_all, stats=mono_bvc_f, nperm=10000, minSize=15, maxSize=300)
mac_res <- fgsea(pathways=pathways_all, stats=mac_bvc_f, nperm=10000, minSize=15, maxSize=300)

In [ ]:
# write all results to file
mono_f <- mono_res
mono_f$leadingEdge <- vapply(mono_f$leadingEdge, paste, collapse = ", ", character(1L))
mac_f <- mac_res
mac_f$leadingEdge <- vapply(mac_f$leadingEdge, paste, collapse = ", ", character(1L))

write.table(mono_f, file="Aug20_BLZ_data/monocyte_fgsea_results_full_Aug20.txt",
            sep="\t", col.names=T, row.names=F, quote=F)
write.table(mac_f, file="Aug20_BLZ_data/macrophage_fgsea_results_full_Aug20.txt",
            sep="\t", col.names=T, row.names=F, quote=F)

In [ ]:
# label pathways a/w immune activation
mono_plot <- mono_res[ES>0][order(-NES)][padj<0.05,]
mono_plot$fn <- "no"

mono_ias <- read.delim("Aug20_BLZ_data/mono_imm-act-list_Aug20.txt", header=F)$V1
mono_plot$fn[(mono_plot$pathway %in% mono_ias)] <- "immune activation"

In [ ]:
mac_plot <- mac_res[ES>0][order(-NES)][padj<0.05,]
mac_plot$fn <- "no"

mac_ias <- read.delim("Aug20_BLZ_data/mac_imm-act-list_Aug20.txt", header=F)$V1
mac_plot$fn[(mac_plot$pathway %in% mac_ias)] <- "immune activation"

In [ ]:
# plot NES vs. -log10(p) for immune activation pathways (FDR<0.05) against all other pathways with NES>0
## for monos
mono_pos <- mono_res[ES>0]
mono_pos$fn <- 'non-immune'

## for macs
mac_pos <- mac_res[ES>0]
mac_pos$fn <- 'non-immune'

## mark immune pathways as "immune activation"
mono_pos$fn[(mono_pos$pathway %in% mono_ias)] <- "immune activation"
mac_pos$fn[(mac_pos$pathway %in% mac_ias)] <- "immune activation"

# define names of pathways to label in graph
mono_pos$name <- ''
mono_pos[mono_pos$pathway == 'GO:0006955_immune_response',]$name <- 'Immune response'
mono_pos[mono_pos$pathway == 'GO:0002443_leukocyte_mediated_immunity',]$name <- 'Leukocyte immunity'
mono_pos[mono_pos$pathway == 'GO:0002456_T_cell_mediated_immunity',]$name <- 'T cell mediated immunity'

# define names of pathways to label in graph
mac_pos$name <- ''
mac_pos[mac_pos$pathway == 'GO:0046649_lymphocyte_activation',]$name <- 'Lymphocyte activation'
mac_pos[mac_pos$pathway == 'GO:0002720_positive_regulation_of_cytokine_production_involved_in_immune_response',]$name <- 'Cytokine induction'
mac_pos[mac_pos$pathway == 'GO:0019882_antigen_processing_and_presentation',]$name <- 'Antigen processing/presentation'
mac_pos[mac_pos$pathway == 'GO:0034341_response_to_interferon-gamma',]$name <- 'Response to interferon-gamma'

momac_pos <- rbind(mono_pos, mac_pos)
colors <- c('immune activation'='#000000', 'non-immune'='#708090')

# make the plot
scatter <- ggplot() +
  theme_cowplot() + 
  coord_fixed(ratio = 1) +
  theme(text=element_text(family="Myriad Pro")) + 
  # plot significant immune activation pathways + label selected pathways
  geom_point(data=momac_pos, aes(x=-log10(pval), y=NES, color=momac_pos$fn,
                                 # non-immune pathways keep transparent
                                 alpha=(momac_pos$fn=="non-immune")), size=1) +
  scale_alpha_manual(values=c(1, 0.1), guide=FALSE) +
  scale_color_manual(values=colors) +
  geom_text_repel(data=subset(momac_pos, length(name)>0), aes(x=-log10(pval),y=NES,label=name),
                  # size=4, nudge_x=-0.2, segment.size=0.2, family="Myriad Pro") +
                  size=4, nudge_x=-1, segment.size=0.2, family="Myriad Pro") +
  theme(axis.title.x=element_text(size=rel(1.1)), axis.title.y=element_text(size=rel(1.1))) +
  theme(legend.position = "none")
  labs(x="-log10(p-value)", y="NES (Blz vs. Ctl)") +
  theme(axis.text.y=element_text(size=rel(1)), axis.text.x=element_text(size=rel(1)))
ggsave(scatter, file="Aug20_BLZ_data/Scatter_PosNES-Pvalue-MoMacs_Immune_0.05_LABELED_Aug20.png", width=7.42, height=5.96)